## Deep Learning Para Aplicações de Inteligência Artificial
### Fine-Tunin de um Modelo Transformer Para Classificação de Sentimento


In [ ]:
# Imports
import math
import nltk
import spacy
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow import keras
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import TextVectorization
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

AttributeError: module 'inspect' has no attribute 'ArgSpec'

Carregando os Dados de Texto

In [ ]:
# Loading train data
dados_dsa_treino = pd.read_csv('dados_treino.txt', header = None, delimiter = ';')


In [ ]:
# Loading test data
dados_dsa_teste = pd.read_csv('dados_teste.txt', header = None, delimiter = ';')

In [ ]:
# Renaming
dados_dsa_teste = dados_dsa_teste.rename(columns = {0 : 'texto', 1 : 'sentimento'})
dados_dsa_treino = dados_dsa_treino.rename(columns  = {0: 'texto', 1 : 'sentimento'})

In [ ]:
# Shape
print(dados_dsa_teste.shape)
# Sample
dados_dsa_treino.head()

(2000, 2)


,texto,sentimento
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [ ]:
# Shape
print(dados_dsa_treino.shape)
# Sample
dados_dsa_teste.head()

(16000, 2)


,texto,sentimento
0,i feel like my only role now would be to tear ...,sadness
1,i feel just bcoz a fight we get mad to each ot...,anger
2,i feel like reds and purples are just so rich ...,joy
3,im not sure the feeling of loss will ever go a...,sadness
4,i feel like ive gotten to know many of you thr...,joy


In [ ]:
# Sentiment count train data
dados_dsa_treino['sentimento'].value_counts()

sentimento
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [ ]:
# Sentiment count test data
dados_dsa_teste['sentimento'].value_counts()

sentimento
joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: count, dtype: int64

### Pré- processamento dos dados
https://spacy.io

In [ ]:
# !python -m spacy download en_core_web_md -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Carrega o dicionário
dsa_nlp = spacy.load('en_core_web_md')

In [ ]:
# Create a function that receives a text as parameter
def das_preprocessa_text(text):

    #Preprocess the text using a dictionaty
    doc = dsa_nlp(text)

    # Create a list with tokens in lower case and without white spaces and stopwords
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

    # Return preprocessed tokens as a unique line, joining by white spaces
    return ' '.join(tokens)

In [ ]:
# Applying the function to train data
dados_dsa_treino['texto_preprocessado'] = dados_dsa_treino['texto'].apply(das_preprocessa_text)

In [ ]:
# Applying the function to test data
dados_dsa_teste['texto_preprocessado'] = dados_dsa_teste['texto'].apply(das_preprocessa_text)

In [ ]:
# Sample
dados_dsa_treino.head()

,texto,sentimento,texto_preprocessado
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


Version 1 - Fully Connecteed Neural NEtwork Arquiteture

Step1: Using TF-IDF to vetorize

In [ ]:
# Vectorization
dsa_tfidf = TfidfTransformer(max_df = 0.95, min_df = 2, stop_words = 'english')

In [ ]:
# Apply vetorizarion
dados_treino_tfidf = dsa_tfidf.fit_transform(dados_dsa_treino['texto_processado'])
dados_teste_tfidf = dsa_tfidf.tranfomr(dados_dsa_teste['texto_processado'])

In [ ]:
dados_treino_tfidf.shape

In [ ]:
type(dados_treino_tfidf)